In [ ]:
!wget http://www.eecs.yorku.ca/~bil/Datasets/for-rerec.tar.gz

--2021-12-03 06:40:29--  http://www.eecs.yorku.ca/~bil/Datasets/for-rerec.tar.gz
Resolving www.eecs.yorku.ca (www.eecs.yorku.ca)... 130.63.94.24
Connecting to www.eecs.yorku.ca (www.eecs.yorku.ca)|130.63.94.24|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.eecs.yorku.ca/~bil/Datasets/for-rerec.tar.gz [following]
--2021-12-03 06:40:29--  https://www.eecs.yorku.ca/~bil/Datasets/for-rerec.tar.gz
Connecting to www.eecs.yorku.ca (www.eecs.yorku.ca)|130.63.94.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1633453504 (1.5G) [application/x-tar]
Saving to: ‘for-rerec.tar.gz’

for-rerec.tar.gz    100%[===================>]   1.52G  93.2MB/s    in 17s     

2021-12-03 06:40:47 (90.2 MB/s) - ‘for-rerec.tar.gz’ saved [1633453504/1633453504]



In [ ]:
!tar -xvf for-rerec.tar.gz

Выходные данные были обрезаны до нескольких последних строк (5000).
for-rerecorded/training/fake/recording2061.wav_norm_mono.wav
for-rerecorded/training/fake/recording2062.wav_norm_mono.wav
for-rerecorded/training/fake/recording2063.wav_norm_mono.wav
for-rerecorded/training/fake/recording2064.wav_norm_mono.wav
for-rerecorded/training/fake/recording2065.wav_norm_mono.wav
for-rerecorded/training/fake/recording2066.wav_norm_mono.wav
for-rerecorded/training/fake/recording2069.wav_norm_mono.wav
for-rerecorded/training/fake/recording2070.wav_norm_mono.wav
for-rerecorded/training/fake/recording2072.wav_norm_mono.wav
for-rerecorded/training/fake/recording2073.wav_norm_mono.wav
for-rerecorded/training/fake/recording2074.wav_norm_mono.wav
for-rerecorded/training/fake/recording2075.wav_norm_mono.wav
for-rerecorded/training/fake/recording2076.wav_norm_mono.wav
for-rerecorded/training/fake/recording2077.wav_norm_mono.wav
for-rerecorded/training/fake/recording2079.wav_norm_mono.wav
for-rerecorded/tr

In [ ]:
import math, random
import torch
import numpy as np
import torchaudio
from IPython.display import Audio
import torchaudio.transforms as T

class AudioUtil():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------
  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)
  @staticmethod
  def resample(aud, newsr):
    sig, sr = aud

    if (sr == newsr):
      # Nothing to do
      return aud

    num_channels = sig.shape[0]
    # Resample first channel
    resig = T.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      retwo = T.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  @staticmethod
  def pad_trunc(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms

    if (sig_len > max_len):
      # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
      # Length of padding to add at the beginning and end of the signal
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len

      # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))

      sig = torch.cat((pad_begin, sig, pad_end), 1)
      
    return (sig, sr)

  @staticmethod
  def spectro_gram(aud, n_mels=256, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    mel = T.MelSpectrogram(
        sr,
        n_fft=n_fft,
        hop_length=393 ,
        win_length=2205,
        n_mels=n_mels,
    )(sig).repeat(3, 1, 1)
    mel = T.AmplitudeToDB(top_db=top_db)(mel)
    # Convert to decibels
    return (mel)

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class VoiceSet(Dataset):
  def __init__(self, voice_df, size=256, channels=1):
    super().__init__()
    self.df = voice_df
    self.size = size
    self.channels = channels
    self.duration = 2000
    self.sr = 44100
    self.channel = 2
    self.shift_pct = 0.4

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    path, label = self.df.iloc[index]
    aud = AudioUtil.open(path)
    reaud = AudioUtil.resample(aud, self.sr)
    dur_aud = AudioUtil.pad_trunc(reaud, self.duration)
    sgram = AudioUtil.spectro_gram(dur_aud, n_mels=224, n_fft=4096, hop_len=None)
    return (sgram, 1 if label == 'fake' else 0)

In [ ]:
import pandas as pd
import os
data = []
sample = 'training'
for file in os.listdir(f'for-rerecorded/{sample}/real'):
  data.append((f'for-rerecorded/{sample}/real/{file}', 'real'))
for file in os.listdir(f'for-rerecorded/{sample}/fake'):
  data.append((f'for-rerecorded/{sample}/fake/{file}', 'fake'))
df = pd.DataFrame(data, columns=['path', 'label'])
df = df.sample(frac=1).reset_index(drop=True)
train_dataset = VoiceSet(df)

data = []
sample = 'validation'
for file in os.listdir(f'for-rerecorded/{sample}/real'):
  data.append((f'for-rerecorded/{sample}/real/{file}', 'real'))
for file in os.listdir(f'for-rerecorded/{sample}/fake'):
  data.append((f'for-rerecorded/{sample}/fake/{file}', 'fake'))
df = pd.DataFrame(data, columns=['path', 'label'])
df = df.sample(frac=1).reset_index(drop=True)
val_dataset = VoiceSet(df)

In [ ]:
train_dataset[0]

(tensor([[[  8.4827,  10.1491,  16.5144,  ...,  14.6194,  11.5690,   5.1323],
          [ 10.0513,  12.7267,  15.7871,  ...,  11.1120,   8.3464,   3.0261],
          [  7.4511,  10.8947,  12.5303,  ...,   4.7200,   2.2489,   0.1558],
          ...,
          [  0.6381,  -2.2037, -13.4575,  ..., -24.3455, -24.3455, -24.3455],
          [ -0.4903,  -3.3192, -14.4562,  ..., -24.3455, -24.3455, -24.3455],
          [ -0.3925,  -3.2219, -14.3632,  ..., -24.3455, -24.3455, -24.3455]],
 
         [[  8.4827,  10.1491,  16.5144,  ...,  14.6194,  11.5690,   5.1323],
          [ 10.0513,  12.7267,  15.7871,  ...,  11.1120,   8.3464,   3.0261],
          [  7.4511,  10.8947,  12.5303,  ...,   4.7200,   2.2489,   0.1558],
          ...,
          [  0.6381,  -2.2037, -13.4575,  ..., -24.3455, -24.3455, -24.3455],
          [ -0.4903,  -3.3192, -14.4562,  ..., -24.3455, -24.3455, -24.3455],
          [ -0.3925,  -3.2219, -14.3632,  ..., -24.3455, -24.3455, -24.3455]],
 
         [[  8.4827,  10.149

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=16, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=16, num_workers=4)
dataloaders = {'train': train_dataloader, 'valid': val_dataloader}
datasets = {'train': train_dataset, 'valid': val_dataset}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# TRAIN
import time
import copy
from tqdm import tqdm_notebook
def train_model(datasets, dataloaders, model, criterion, optimizer, scheduler, num_epochs, device):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print('-' * 10)
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0.0
            running_total = 0.0
            
            for step, (inputs, labels) in tqdm_notebook(enumerate(dataloaders[phase])):
                inputs = inputs.to(device)
                labels = labels.to(device)
               
                # Zero out the grads
                optimizer.zero_grad()
                
                # Forward
                # Track history in train mode
                with torch.set_grad_enabled(phase == 'train'):
                    model = model.to(device)
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) 
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_total += len(labels.data)
                
                if (step + 1) % 100 == 0:
                    print(f'[{step + 1}/{len(dataloaders[phase])}].')
                    print(f'Loss {running_loss / running_total}. Accuracy {running_corrects / running_total}')
            
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / len(datasets[phase])
            epoch_acc = running_corrects.double() / len(datasets[phase])
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torchvision.models as models
from torchaudio.models import ConvTasNet
import torch.nn as nn

###Resnet101

In [ ]:
model = models.resnet101(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,2)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

criterion = nn.CrossEntropyLoss()
num_epochs = 5

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [ ]:
trained_model = train_model(datasets, dataloaders,
                            model, criterion,
                            optimizer, scheduler,
                            num_epochs, device)

Epoch 0/4
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

[100/160].
Loss 0.08687194412341341. Accuracy 0.9665624499320984
train Loss: 0.0638 Acc: 0.9755


0it [00:00, ?it/s]

...

valid Loss: 0.0091 Acc: 0.9964

Epoch 4/4
----------


0it [00:00, ?it/s]

[100/160].
Loss 0.00040627359278005314. Accuracy 1.0
train Loss: 0.0003 Acc: 1.0000


0it [00:00, ?it/s]

valid Loss: 0.0081 Acc: 0.9978

Training complete in 65m 7s
Best val Acc: 0.9978


In [ ]:
torch.save(model, 'audio_resnet.pt')

###Transformer

In [ ]:
!pip install vision_transformer_pytorch

In [ ]:
from vision_transformer_pytorch import VisionTransformer
model = VisionTransformer.from_name('ViT-B_16', num_classes=2, image_size=224)

In [ ]:
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

criterion = nn.CrossEntropyLoss()
num_epochs = 3

In [ ]:
trained_model = train_model(datasets, dataloaders,
                            model, criterion,
                            optimizer, scheduler,
                            num_epochs, device)

Epoch 0/2
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

[100/2552].
Loss 0.7486130362749099. Accuracy 0.5349999666213989
[200/2552].
Loss 0.7619471752643585. Accuracy 0.5062499642372131
[300/2552].
Loss 0.7611738794048627. Accuracy 0.512499988079071
[400/2552].
Loss 0.7563860075175762. Accuracy 0.5149999856948853
[500/2552].
Loss 0.758197506070137. Accuracy 0.5135000348091125
[600/2552].
Loss 0.7576331725219886. Accuracy 0.5091666579246521
[700/2552].
Loss 0.7563604567306382. Accuracy 0.5089285373687744
[800/2552].
Loss 0.7541984795406461. Accuracy 0.5134375095367432
[900/2552].
Loss 0.7545673256119092. Accuracy 0.5155555605888367
[1000/2552].
Loss 0.7517775246798992. Accuracy 0.5170000195503235
[1100/2552].
Loss 0.7509052959355441. Accuracy 0.517727255821228
[1200/2552].
Loss 0.7478074458489815. Accuracy 0.5218750238418579
[1300/2552].
Loss 0.7467179665198693. Accuracy 0.5228846073150635
[1400/2552].
Loss 0.7463287594488689. Accuracy 0.5219642519950867
[1500/2552].
Loss 0.7448318871458371. Accuracy 0.5214999914169312
[1600/2552].
Loss 0.74

0it [00:00, ?it/s]

[100/561].
Loss 0.7332828581333161. Accuracy 0.5149999856948853
[200/561].
Loss 0.7144394677877426. Accuracy 0.5462499856948853
[300/561].
Loss 0.7137001391251881. Accuracy 0.5441666841506958
[400/561].
Loss 0.7072729694843293. Accuracy 0.5537499785423279
[500/561].
Loss 0.7037875332832336. Accuracy 0.5555000305175781
valid Loss: 0.7016 Acc: 0.5593

Epoch 1/2
----------


0it [00:00, ?it/s]

[100/2552].
Loss 0.6991731214523316. Accuracy 0.5575000047683716
[200/2552].
Loss 0.7048554956912995. Accuracy 0.5537499785423279
[300/2552].
Loss 0.7029887436827024. Accuracy 0.54666668176651
[400/2552].
Loss 0.700776324570179. Accuracy 0.5562499761581421
[500/2552].
Loss 0.6983145473003387. Accuracy 0.5595000386238098
[600/2552].
Loss 0.6985134694973628. Accuracy 0.5570833683013916
[700/2552].
Loss 0.6952791486467634. Accuracy 0.5596428513526917
[800/2552].
Loss 0.6933902110159397. Accuracy 0.5662499666213989


###EfficientNet

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=9bd086d555a2b035e48a4490eae4b4d3dcfb00e0d2172b9697bff6bc87548e4d
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=2)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7


In [ ]:
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

criterion = nn.CrossEntropyLoss()
num_epochs = 3

In [ ]:
trained_model = train_model(datasets, dataloaders,
                            model, criterion,
                            optimizer, scheduler,
                            num_epochs, device)

Epoch 0/2
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

[100/638].
Loss 0.3175870669260621. Accuracy 0.859375
[200/638].
Loss 0.22179615779779852. Accuracy 0.9081249833106995
[300/638].
Loss 0.17438903647474946. Accuracy 0.9291666746139526
[400/638].
Loss 0.1441316580941202. Accuracy 0.9439062476158142
[500/638].
Loss 0.12408144325041212. Accuracy 0.952625036239624
[600/638].
Loss 0.1103523767722072. Accuracy 0.9582291841506958
train Loss: 0.1046 Acc: 0.9606


0it [00:00, ?it/s]

...

[100/141].
Loss 0.06629677345143137. Accuracy 0.9975000023841858
valid Loss: 0.0579 Acc: 0.9960

Epoch 2/2
----------


0it [00:00, ?it/s]

[100/638].
Loss 0.0034938184355996784. Accuracy 0.9993749856948853
[200/638].
Loss 0.0029385078694167534. Accuracy 0.9993749856948853
[300/638].
Loss 0.002402965108979818. Accuracy 0.99958336353302
[400/638].
Loss 0.003239982819729903. Accuracy 0.9992187023162842
[500/638].
Loss 0.0030456933264849796. Accuracy 0.99937504529953
[600/638].
Loss 0.002684921214881797. Accuracy 0.9994791746139526
train Loss: 0.0026 Acc: 0.9995


0it [00:00, ?it/s]

[100/141].
Loss 0.037563102016229095. Accuracy 0.9987499713897705
valid Loss: 0.0332 Acc: 0.9973

Training complete in 72m 33s
Best val Acc: 0.9973


In [ ]:
trained_model = train_model(datasets, dataloaders,
                            model, criterion,
                            optimizer, scheduler,
                            1, device)

Epoch 0/0
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

[100/638].
Loss 0.0023727157026041824. Accuracy 0.9993749856948853
[200/638].
Loss 0.001861595992513685. Accuracy 0.9993749856948853
[300/638].
Loss 0.0015132966798410052. Accuracy 0.99958336353302
[400/638].
Loss 0.0013316489942553744. Accuracy 0.9996874928474426
[500/638].
Loss 0.0011889641917732661. Accuracy 0.999750018119812
[600/638].
Loss 0.0010635713782448874. Accuracy 0.99979168176651
train Loss: 0.0010 Acc: 0.9998


0it [00:00, ?it/s]

[100/141].
Loss 0.058953657736501554. Accuracy 0.9987499713897705
valid Loss: 0.0480 Acc: 0.9973

Training complete in 24m 33s
Best val Acc: 0.9973
